# Emergency Networking API - Connection and data fetch

In [ ]:
# Import modules

In [ ]:
# Setup main variables

## Data Ingest

In [ ]:
# API call

# Data sample

## Data stats and charts

In [ ]:
# Variable exploration and quality analysis

In [ ]:
# Basic stats and charts

## Data modeling and exporting

In [ ]:
# Table expands and key propagation

In [ ]:
# CSV export